In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# your first tensorflow program

a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)

with tf.Session() as sess:
    print(sess.run(x))

5


In [5]:
# To visualize with graph with tensorboard

a = tf.constant(3, name='a')
b = tf.constant(5, name='b')

x = tf.add(a, b, name='add')
# create the summary writer after creating the graph and before running the session
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run(x))
writer.close()

8


#### defining a constant

    tf.constant(value, 
            dtype=None,
            shape=None,
            name='Const',
            verify_shape=False)

In [7]:
# Broadcasting also runs in tensorflow, very similar to numpy array

a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
prod = tf.multiply(a, b, name='prod')

writer = tf.summary.FileWriter('./graphs/', tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run(prod))


[[0 2]
 [4 6]]


### creating a tensors filled with certain value 

    tf.zeros(shape, dtype=tf.float32, name=None)

    tf.zeros_like(input_tensor)
    
    tf.ones(shape, dtype=tf.float32, name=None)
    
    tf.ones_like(input_tensor, dtype=None, name=None, optimize=True)
    
    tf.fill(dims, value, name=None) EX- tf.fill([2, 3], 8) ==> [[8, 8, 8], [8, 8, 8]]

### constants as sequences

    1. tf.lin_space(start, stop, num, name=None) 
    EX - tf.lin_space(10.0, 13.0, 4) ==> [10., 11., 12., 13.]
    
    2. tf.range(start, limit=None, delta=1, dtype=None, name='range')
    Ex - tf.range(3, 18, 3) ==> [3, 6, 9, 12, 15]
    Ex - tf.range(5) ==> [0, 1, 2, 3, 4] 
    *Note - Tensor objects are not iterable*
    
    3. tf.random module generates randomly generated constants based on the seed tf.set_random_seed(seed)

In [3]:
# tensorflow seamlessly integrates with numpy array

tf.int32 == np.int32

True

In [4]:
tf.ones([2, 2], np.float32)

<tf.Tensor 'ones:0' shape=(2, 2) dtype=float32>

In [5]:
sess = tf.Session()
a = tf.zeros([2, 3], np.int32)
print(type(a))
a = sess.run(a)
print(a, type(a))

<class 'tensorflow.python.framework.ops.Tensor'>
[[0 0 0]
 [0 0 0]] <class 'numpy.ndarray'>


#### Constants are stored in graph definition and graph definition is stored in protobuf (protocol buffers, which is google's language neutral, platform neutral, extensible mechanism for serializing data much like XML but smaller, faster, simpler. But when using constants with makes loading graphs much expensive when constants are big. Thus only use constants for primitive types and use variables or readers for more data that requires more memory

In [6]:
# printing out the graph def

my_const = tf.constant([1.0, 2.0], name="my_const")

with tf.Session() as sess:
    print(sess.graph.as_graph_def())

node {
  name: "ones/shape_as_tensor"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "ones/Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "ones"
  op: "Fill"
  input: "ones/shape_as_tensor"
  input: "ones/Const"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "index_type"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "zeros/shape_as_tensor"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"


### Variables

In [9]:
# create variables with tf.Variable

s = tf.Variable(2, name='scalar')
m = tf.Variable([[0, 1], [2, 3]], name='matrix')
w = tf.Variable(tf.zeros([784, 10]))

# also variables can be created by using tf.get_variable
sg = tf.get_variable("scalar2", initializer=tf.constant(2))
mg = tf.get_variable("matrix2", initializer=tf.constant([[0, 1], [2, 4]]))
wg = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

In [15]:
s, print(s.value())

Tensor("scalar_3/read:0", shape=(), dtype=int32)


(<tf.Variable 'scalar_3:0' shape=() dtype=int32_ref>, None)

### Why tf.const and tf.Variable? it is because tf.const is an op where as tf.Variable is a class with many ops.

    tf.Variable(...) holds several ops which are 
    x.initializer # init op
    x.value() # read op
    x.assign() # write op
    x.assign_add() # and more
    
    #### If you try to access uninitialized variables there will be an exception, thus always initialize the variables before accessing them and the best way to do that is to use global variables initializer
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
    Since Initializer is an op, it must be executed within the context of a session. Also only a subset of variables can be initialized as follows or if needed only a single variable can also be initialized
    
    # initializing set of variables
    with tf.Session() as sess:
        sess.run(tf.variables_initializer([a, b]))
        
    # initializing single variable
    with tf.Session() as sess:
        sess.run(W.initializer)

In [16]:
# evaluating a variable
# W_rand is a random 700x100 variable object

w_rand = tf.Variable(tf.truncated_normal([700, 100]))
with tf.Session() as sess:
    sess.run(w_rand.initializer)
    print(w_rand.eval()) # similar to print(sess.run(W_rand))

[[ 1.834228    0.15182053 -0.6002811  ... -0.21675114  1.2316219
  -0.30490774]
 [ 0.02363384 -0.6221866  -0.89841247 ...  1.0145808  -0.4358228
  -0.38684115]
 [ 0.08975025  0.93958086  0.196388   ... -1.2610238  -1.2481776
   0.8720634 ]
 ...
 [-0.24828453 -0.32498893 -0.48279732 ...  1.2932884  -1.4542863
   0.26246408]
 [-0.1928998   0.92277104  0.14586735 ...  0.12521408  0.5507113
  -0.15304334]
 [-0.4665528  -0.724336   -0.02024512 ...  0.44051266  0.46336374
  -0.6554293 ]]


In [17]:
## info regarding the tf.Variable.assign()

w1 = tf.Variable(10)
w1.assign(33)

with tf.Session() as sess:
    sess.run(w1.initializer)
    print(w1.eval()) 

10


#### The above cell prints 10 instead of 33 because tf.variable.assign() is an op and should be in the context of a session as all other ops also the assign op actually initializes the variables so those variables need not be initialized again

In [20]:
w2 = tf.Variable(45)
assgin_op = w2.assign(33)

with tf.Session() as sess:
    sess.run(w2.initializer)
    sess.run(assgin_op)
    print(w2.eval())

33


#### Each session maintains its own set of variables

w = tf.Variable(10)

s1 = tf.Session()
s2 = tf.Session()

s1.run(w.initializer)
s2.run(w.initializer)

print(s1.run(w.assign_add(10))) ==> 20
print(s2.run(w.assign_sub(2))) ==> 8

print(s1.run(w.assign_add(40))) ==> 60
print(s2.run(w.assign_sub(50))) ==> -42

s1.close()
s2.close()

#### sometimes when we have two or more independent ops and you'd like to specify which op should run first, then use tf.Graph.control_dependencies(control_inputs)

#### Structure of a tf program

    1. Assemble a graph
    2. Use a session to execute the operations of the graph (this is very much like a general function f(x, y) = 2 * x + y where x and y are actualy placeholders)

In [21]:
# using placeholders tf.placeholder(dtype, shape=None, name=None)

# create a placeholder for a vector of 3 elements, type tf.float32
a = tf.placeholder(tf.float32, shape=[3])

b = tf.constant([5, 5, 5], tf.float32)

# use a placeholder as you would use a constant  or a variable
c = a + b # short for tf.add(a, b)

with tf.Session() as sess:
    
    #print(sess.run(c)) # would throw an error since a doesn't have a value, 
    #so we have to supplement values to placeholder using dict
    
    print(sess.run(c, feed_dict={a: [1, 2, 3]})) # tensor a is the key, not the string 'a'
    
    


[6. 7. 8.]


#### usually it's best practice to define shape in the tf.placeholder since it will be easier later on for debugging and what if we want to feed multiple data points in 

    with tf.Session() as sess:
        for a_value in list_of_values_for_a:
            print(sess.run(c, feed_dict={a: a_value}))
            

#### To know whether a tensor is feedable or not use tf.Graph.is_feedable(tensor) which would be true if and only if tensor is feedable


#### Lazy loading - Defer creating / initializing an object until it is needed and never use lazy loading since it bloats up the graph and it's very expensive to pass around the graph, if certain op is executed multiple times